In [1]:
import os
import pickle
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import pearsonr

In [2]:
def age_debiasing(predicted_ages, chronological_ages):
    
    predicted_ages = predicted_ages.to_numpy()
    chronological_ages = chronological_ages.to_numpy()
    
    ones = np.ones(len(chronological_ages))
    chronological_ages_squared = np.square(chronological_ages)
    chronological_ages_confounds = np.array([ones, chronological_ages])
    
    ones_predicted = np.ones(len(predicted_ages))
    predicted_ages_squared = np.square(predicted_ages)
    predicted_ages_confounds = np.array([ones_predicted, predicted_ages])
    
    predicted_ages = np.reshape(predicted_ages, (len(predicted_ages), 1) )
    beta = np.dot( np.linalg.pinv(chronological_ages_confounds.T), predicted_ages )

    corrected_predicted_ages = (predicted_ages.flatten() - beta[0])/beta[1]
    
    return corrected_predicted_ages

In [3]:
T1_nonlinear="1"
T1_linear="2"
jacobian="56"
vbm="4"

T2_nonlinear="5"
T2_lesions="57"
swi="6"

rsfmri_0="7"
rsfmri_1="8"
rsfmri_2="9"
rsfmri_3="10"
rsfmri_4="11"
rsfmri_5="12"
rsfmri_6="13"
rsfmri_7="14"
rsfmri_8="15"
rsfmri_9="16"
rsfmri_10="17"
rsfmri_11="18"
rsfmri_12="19"
rsfmri_13="20"
rsfmri_14="21"
rsfmri_15="22"
rsfmri_16="23"
rsfmri_17="24"
rsfmri_18="25"
rsfmri_19="26"
rsfmri_20="27"
rsfmri_21="28"
rsfmri_22="29"
rsfmri_23="30"
rsfmri_24="31"

tfmri_1="32"
tfmri_2="33"
tfmri_5="34"

tfmri_c_1="35"
tfmri_c_2="36"
tfmri_c_5="37"

tracts="3"

tbss_FA_s="38"
tbss_ICVF_s="39"
tbss_ISOVF_s="40"
tbss_L1_s="41"
tbss_L2_s="42"
tbss_L3_s="43"
tbss_MD_s="44"
tbss_MO_s="45"
tbss_OD_s="46"

tbss_FA="47"
tbss_ICVF="48"
tbss_ISOVF="49"
tbss_L1="50"
tbss_L2="51"
tbss_L3="52"
tbss_MD="53"
tbss_MO="54"
tbss_OD="55"

dfs = [

T1_nonlinear,
T1_linear,
jacobian,
vbm,

T2_nonlinear,
T2_lesions,
swi,

rsfmri_0,
rsfmri_1,
rsfmri_2,
rsfmri_3,
rsfmri_4,
rsfmri_5,
rsfmri_6,
rsfmri_7,
rsfmri_8,
rsfmri_9,
rsfmri_10,
rsfmri_11,
rsfmri_12,
rsfmri_13,
rsfmri_14,
rsfmri_15,
rsfmri_16,
rsfmri_17,
rsfmri_18,
rsfmri_19,
rsfmri_20,
rsfmri_21,
rsfmri_22,
rsfmri_23,
rsfmri_24,

tfmri_1,
tfmri_2,
tfmri_5,

tfmri_c_1,
tfmri_c_2,
tfmri_c_5,

tracts,

tbss_FA_s,
tbss_ICVF_s,
tbss_ISOVF_s,
tbss_L1_s,
tbss_L2_s,
tbss_L3_s,
tbss_MD_s,
tbss_MO_s,
tbss_OD_s,

tbss_FA,
tbss_ICVF,
tbss_ISOVF,
tbss_L1,
tbss_L2,
tbss_L3,
tbss_MD,
tbss_MO,
tbss_OD
]

names = [

'T1_nonlinear',
'T1_linear',
'jacobian',
'vbm',

'T2_nonlinear',
'T2_lesions',
'swi',

'rsfmri_0',
'rsfmri_1',
'rsfmri_2',
'rsfmri_3',
'rsfmri_4',
'rsfmri_5',
'rsfmri_6',
'rsfmri_7',
'rsfmri_8',
'rsfmri_9',
'rsfmri_10',
'rsfmri_11',
'rsfmri_12',
'rsfmri_13',
'rsfmri_14',
'rsfmri_15',
'rsfmri_16',
'rsfmri_17',
'rsfmri_18',
'rsfmri_19',
'rsfmri_20',
'rsfmri_21',
'rsfmri_22',
'rsfmri_23',
'rsfmri_24',

'tfmri_1',
'tfmri_2',
'tfmri_5',

'tfmri_c_1',
'tfmri_c_2',
'tfmri_c_5',

'tracts',

'tbss_FA_s',
'tbss_ICVF_s',
'tbss_ISOVF_s',
'tbss_L1_s',
'tbss_L2_s',
'tbss_L3_s',
'tbss_MD_s',
'tbss_MO_s',
'tbss_OD_s',

'tbss_FA',
'tbss_ICVF',
'tbss_ISOVF',
'tbss_L1',
'tbss_L2',
'tbss_L3',
'tbss_MD',
'tbss_MO',
'tbss_OD'
]

dfs1, dfs2, dfs3 = [], [], []

dfs1 = dfs.copy()
dfs2 = dfs.copy()
dfs3 = dfs.copy()

for idx in range(len(dfs)):
    
    dfs1[idx] = '../predictions/AM5-' + dfs1[idx] + '_predictions/output_statistics_train.csv'
    dfs2[idx] = '../predictions/AM6-' + dfs2[idx] + '_predictions/output_statistics_train.csv'
    dfs3[idx] = '../predictions/AM7-' + dfs3[idx] + '_predictions/output_statistics_train.csv'
    
for idx in range(len(dfs)):
    dfs1[idx] = pd.read_csv(dfs1[idx])
    dfs2[idx] = pd.read_csv(dfs2[idx])
    dfs3[idx] = pd.read_csv(dfs3[idx])

In [4]:
for idx in range(len(dfs)):
    dfs[idx] = pd.merge(dfs1[idx], dfs2[idx], on='Unnamed: 0', how='inner')
    dfs[idx] = pd.merge(dfs[idx], dfs3[idx], on='Unnamed: 0', how='inner')
    
    dfs[idx].rename({
        'target_age_x': 'target_age_original',
        'output_age_x': 'output_age_1',
        'age_delta_x': 'age_delta_1',
        'loss_x': 'loss_1',
        'target_age_y': 'target_age_2',
        'output_age_y': 'output_age_2',
        'age_delta_y': 'age_delta_2',
        'loss_y': 'loss_2',
        'target_age': 'target_age_3',
        'output_age': 'output_age_3',
        'age_delta': 'age_delta_3',
        'loss': 'loss_3',
    }, axis=1, inplace=True)
    
    dfs[idx].drop(['target_age_2', 'target_age_3'], axis=1, inplace=True)
    
    dfs[idx].rename({
        'target_age_original': 'target_age',
    }, axis=1, inplace=True)
    
    dfs[idx]['mean_output_age'] = dfs[idx][['output_age_1', 'output_age_2', 'output_age_3']].mean(axis=1)
    dfs[idx]['mean_age_delta'] = dfs[idx]['mean_output_age'] - dfs[idx]['target_age']

In [5]:
for idx in range(len(dfs)):    
    dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].mean_output_age, dfs[idx].target_age) 
    dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

In [6]:
print("{:13} : {:15}  |  {:13}  |  {:13}  |  {:13}".format('Modality', 'Run1', "Run2", "Run3", "MEAN"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} : {:7.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  MEAN --- {:.3f} ± {:.3f}".format(
        names[idx],
        np.abs(dfs[idx].age_delta_1).mean(), np.abs(dfs[idx].age_delta_1).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_2).mean(), np.abs(dfs[idx].age_delta_2).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_3).mean(), np.abs(dfs[idx].age_delta_3).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : Run1             |  Run2           |  Run3           |  MEAN         
-----------------------------------------------------------------------------------------------
T1_nonlinear  :   1.727 ± 0.014  |  0.921 ± 0.008  |  1.047 ± 0.009  |  MEAN --- 0.895 ± 0.007
T1_linear     :   2.292 ± 0.019  |  1.121 ± 0.009  |  0.556 ± 0.005  |  MEAN --- 0.960 ± 0.008
jacobian      :   2.954 ± 0.025  |  3.143 ± 0.026  |  0.433 ± 0.004  |  MEAN --- 1.872 ± 0.016
vbm           :   1.389 ± 0.012  |  0.785 ± 0.007  |  1.919 ± 0.016  |  MEAN --- 1.027 ± 0.009
T2_nonlinear  :   1.663 ± 0.014  |  1.919 ± 0.016  |  0.794 ± 0.007  |  MEAN --- 1.181 ± 0.010
T2_lesions    :   3.757 ± 0.031  |  3.719 ± 0.031  |  3.479 ± 0.030  |  MEAN --- 3.552 ± 0.030
swi           :   2.573 ± 0.022  |  0.659 ± 0.006  |  0.811 ± 0.007  |  MEAN --- 0.842 ± 0.007
rsfmri_0      :   3.953 ± 0.033  |  4.472 ± 0.035  |  3.306 ± 0.027  |  MEAN --- 3.574 ± 0.029
rsfmri_1      :   3.874 ± 0.031  |  3.802 ± 0.031  |  3.72

In [7]:
print("{:13} : {:14}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : MEAN (MAE)      |  MEAN DECONF    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.895 ± 0.007  |   0.846 ± 0.007 
T1_linear     :  0.960 ± 0.008  |   0.934 ± 0.008 
jacobian      :  1.872 ± 0.016  |   1.894 ± 0.016 
vbm           :  1.027 ± 0.009  |   0.995 ± 0.008 
T2_nonlinear  :  1.181 ± 0.010  |   1.130 ± 0.010 
T2_lesions    :  3.552 ± 0.030  |   4.382 ± 0.036 
swi           :  0.842 ± 0.007  |   0.821 ± 0.007 
rsfmri_0      :  3.574 ± 0.029  |   4.047 ± 0.034 
rsfmri_1      :  3.535 ± 0.029  |   3.948 ± 0.033 
rsfmri_2      :  3.591 ± 0.028  |   3.667 ± 0.031 
rsfmri_3      :  3.989 ± 0.032  |   4.829 ± 0.040 
rsfmri_4      :  3.618 ± 0.030  |   4.231 ± 0.036 
rsfmri_5      :  3.241 ± 0.026  |   3.578 ± 0.030 
rsfmri_6      :  4.348 ± 0.034  |   5.259 ± 0.044 
rsfmri_7      :  4.846 ± 0.038  |   7.193 ± 0.059 
rsfmri_8      :  3.472 ± 0.028  |   3.532 ± 0.030 
rsfmri_9      :  3.611 ± 0.030  |   4

In [8]:
mae_weighting = 37.137671232876755 # for male subjects
# mae_weighting = 36.28150684931529 # for female subjects


print("{:13} : {:14}  |  {:15}  |  {:7}  |  {:13} ".format('Modality', 'MAE', "RMSE", 
                                                            "CORR (R)", "MAE weighted"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:5.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0],
        np.abs(dfs[idx].mean_age_delta).mean() / mae_weighting,
    )
         )

Modality      : MAE             |  RMSE             |  CORR (R)  |  MAE weighted  
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.895 ± 0.007  |   1.121 ± 0.140   |   0.990   |   0.024 
T1_linear     :  0.960 ± 0.008  |   1.205 ± 0.151   |   0.989   |   0.026 
jacobian      :  1.872 ± 0.016  |   2.353 ± 0.296   |   0.955   |   0.050 
vbm           :  1.027 ± 0.009  |   1.291 ± 0.163   |   0.987   |   0.028 
T2_nonlinear  :  1.181 ± 0.010  |   1.484 ± 0.186   |   0.983   |   0.032 
T2_lesions    :  3.552 ± 0.030  |   4.466 ± 0.562   |   0.814   |   0.096 
swi           :  0.842 ± 0.007  |   1.056 ± 0.133   |   0.991   |   0.023 
rsfmri_0      :  3.574 ± 0.029  |   4.417 ± 0.532   |   0.833   |   0.096 
rsfmri_1      :  3.535 ± 0.029  |   4.379 ± 0.529   |   0.839   |   0.095 
rsfmri_2      :  3.591 ± 0.028  |   4.387 ± 0.517   |   0.856   |   0.097 
rsfmri_3      :  3.989 ± 0.032  |   4.932 ± 0.593   |   0.784   |   0.1

In [9]:
maes=[]
prs = []
for idx in range(len(dfs)):
    mae = np.abs(dfs[idx].mean_age_delta).mean()
    maes.append(mae)
    pr = pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0]
    prs.append(pr)
    
print(pearsonr(maes, prs)[0])

-0.9717843813102938


In [10]:
print("{:13} : {:14}  |  {:15}  |  {:13} ".format('Modality', 'MAE (DECON)', "RMSE (DECON)", "PR (DECON)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  RMSE (DECON)     |  PR (DECON)    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.846 ± 0.007  |   1.065 ± 0.134   |   0.990 
T1_linear     :  0.934 ± 0.008  |   1.169 ± 0.146   |   0.989 
jacobian      :  1.894 ± 0.016  |   2.372 ± 0.297   |   0.955 
vbm           :  0.995 ± 0.008  |   1.253 ± 0.159   |   0.987 
T2_nonlinear  :  1.130 ± 0.010  |   1.425 ± 0.179   |   0.983 
T2_lesions    :  4.382 ± 0.036  |   5.455 ± 0.677   |   0.814 
swi           :  0.821 ± 0.007  |   1.030 ± 0.130   |   0.991 
rsfmri_0      :  4.047 ± 0.034  |   5.079 ± 0.638   |   0.833 
rsfmri_1      :  3.948 ± 0.033  |   4.965 ± 0.621   |   0.839 
rsfmri_2      :  3.667 ± 0.031  |   4.618 ± 0.582   |   0.856 
rsfmri_3      :  4.829 ± 0.040  |   6.050 ± 0.754   |   0.784 
rsfmri_4      :  4.231 ± 0.036  |   5.332 ± 0.677   |   0.820 
rsfmri_5      :  3.578 ± 0.030  |   4.493 ± 0.566   |   0.862 
rsfmri_6      :

# Saving Dataframe for Later Use

In [11]:
d = {'modality':names, 'dataframe':dfs}
df = pd.DataFrame(data=d)

In [12]:
df['dataframe'].iloc[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23470825,72.307420,68.709730,-3.597688,12.943359,73.018036,0.710616,0.504975,73.407270,1.099852,1.209675,71.711679,-0.595741,72.360083,0.052662
1,23818742,57.839840,57.093216,-0.746624,0.557448,59.144497,1.304657,1.702129,57.383564,-0.456276,0.208188,57.873759,0.033919,57.574667,-0.265173
2,21725930,65.692352,62.648940,-3.043411,9.262348,64.478160,-1.214195,1.474268,64.320274,-1.372077,1.882596,63.815791,-1.876560,63.923556,-1.768796
3,24251841,67.121804,66.739975,-0.381829,0.145793,68.066060,0.944259,0.891626,69.167770,2.045967,4.185980,67.991268,0.869465,68.384932,1.263128
4,21668564,75.721995,74.669510,-1.052485,1.107724,75.039734,-0.682261,0.465480,75.941560,0.219564,0.048208,75.216935,-0.505060,76.105347,0.383352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,21345351,68.506507,69.871380,1.364877,1.862889,68.961060,0.454553,0.206618,70.393020,1.886514,3.558934,69.741820,1.235313,70.255346,1.748839
8180,23866237,71.681849,67.194534,-4.487315,20.135996,71.549890,-0.131961,0.017414,70.523605,-1.158244,1.341529,69.756010,-1.925840,70.270507,-1.411342
8181,23708911,52.711066,55.754790,3.043726,9.264266,53.466724,0.755659,0.571020,52.470910,-0.240156,0.057675,53.897475,1.186409,53.326123,0.615057
8182,22168848,51.871575,50.325333,-1.546243,2.390866,51.601430,-0.270146,0.072979,50.723550,-1.148026,1.317965,50.883438,-0.988138,50.105712,-1.765864


In [13]:
df

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [14]:
type(df.dataframe.iloc[0])

pandas.core.frame.DataFrame

In [15]:
df.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23470825,72.307420,68.709730,-3.597688,12.943359,73.018036,0.710616,0.504975,73.407270,1.099852,1.209675,71.711679,-0.595741,72.360083,0.052662
1,23818742,57.839840,57.093216,-0.746624,0.557448,59.144497,1.304657,1.702129,57.383564,-0.456276,0.208188,57.873759,0.033919,57.574667,-0.265173
2,21725930,65.692352,62.648940,-3.043411,9.262348,64.478160,-1.214195,1.474268,64.320274,-1.372077,1.882596,63.815791,-1.876560,63.923556,-1.768796
3,24251841,67.121804,66.739975,-0.381829,0.145793,68.066060,0.944259,0.891626,69.167770,2.045967,4.185980,67.991268,0.869465,68.384932,1.263128
4,21668564,75.721995,74.669510,-1.052485,1.107724,75.039734,-0.682261,0.465480,75.941560,0.219564,0.048208,75.216935,-0.505060,76.105347,0.383352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,21345351,68.506507,69.871380,1.364877,1.862889,68.961060,0.454553,0.206618,70.393020,1.886514,3.558934,69.741820,1.235313,70.255346,1.748839
8180,23866237,71.681849,67.194534,-4.487315,20.135996,71.549890,-0.131961,0.017414,70.523605,-1.158244,1.341529,69.756010,-1.925840,70.270507,-1.411342
8181,23708911,52.711066,55.754790,3.043726,9.264266,53.466724,0.755659,0.571020,52.470910,-0.240156,0.057675,53.897475,1.186409,53.326123,0.615057
8182,22168848,51.871575,50.325333,-1.546243,2.390866,51.601430,-0.270146,0.072979,50.723550,-1.148026,1.317965,50.883438,-0.988138,50.105712,-1.765864


In [16]:
df.iloc[0].modality

'T1_nonlinear'

In [17]:
type(df.iloc[0].dataframe)

pandas.core.frame.DataFrame

In [18]:
df.to_pickle('male_train.pkl', protocol = 4)

In [19]:
df2 = pd.read_pickle('male_train.pkl')

In [20]:
df2

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [21]:
df2.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23470825,72.307420,68.709730,-3.597688,12.943359,73.018036,0.710616,0.504975,73.407270,1.099852,1.209675,71.711679,-0.595741,72.360083,0.052662
1,23818742,57.839840,57.093216,-0.746624,0.557448,59.144497,1.304657,1.702129,57.383564,-0.456276,0.208188,57.873759,0.033919,57.574667,-0.265173
2,21725930,65.692352,62.648940,-3.043411,9.262348,64.478160,-1.214195,1.474268,64.320274,-1.372077,1.882596,63.815791,-1.876560,63.923556,-1.768796
3,24251841,67.121804,66.739975,-0.381829,0.145793,68.066060,0.944259,0.891626,69.167770,2.045967,4.185980,67.991268,0.869465,68.384932,1.263128
4,21668564,75.721995,74.669510,-1.052485,1.107724,75.039734,-0.682261,0.465480,75.941560,0.219564,0.048208,75.216935,-0.505060,76.105347,0.383352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,21345351,68.506507,69.871380,1.364877,1.862889,68.961060,0.454553,0.206618,70.393020,1.886514,3.558934,69.741820,1.235313,70.255346,1.748839
8180,23866237,71.681849,67.194534,-4.487315,20.135996,71.549890,-0.131961,0.017414,70.523605,-1.158244,1.341529,69.756010,-1.925840,70.270507,-1.411342
8181,23708911,52.711066,55.754790,3.043726,9.264266,53.466724,0.755659,0.571020,52.470910,-0.240156,0.057675,53.897475,1.186409,53.326123,0.615057
8182,22168848,51.871575,50.325333,-1.546243,2.390866,51.601430,-0.270146,0.072979,50.723550,-1.148026,1.317965,50.883438,-0.988138,50.105712,-1.765864


In [22]:
df2.iloc[0].modality

'T1_nonlinear'

In [23]:
type(df2.iloc[0].dataframe)

pandas.core.frame.DataFrame

# Test what happens if debiasing before deconfounding

In [24]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,23470825,72.307420,68.709730,-3.597688,12.943359,73.018036,0.710616,0.504975,73.407270,1.099852,1.209675,71.711679,-0.595741,72.360083,0.052662
1,23818742,57.839840,57.093216,-0.746624,0.557448,59.144497,1.304657,1.702129,57.383564,-0.456276,0.208188,57.873759,0.033919,57.574667,-0.265173
2,21725930,65.692352,62.648940,-3.043411,9.262348,64.478160,-1.214195,1.474268,64.320274,-1.372077,1.882596,63.815791,-1.876560,63.923556,-1.768796
3,24251841,67.121804,66.739975,-0.381829,0.145793,68.066060,0.944259,0.891626,69.167770,2.045967,4.185980,67.991268,0.869465,68.384932,1.263128
4,21668564,75.721995,74.669510,-1.052485,1.107724,75.039734,-0.682261,0.465480,75.941560,0.219564,0.048208,75.216935,-0.505060,76.105347,0.383352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,21345351,68.506507,69.871380,1.364877,1.862889,68.961060,0.454553,0.206618,70.393020,1.886514,3.558934,69.741820,1.235313,70.255346,1.748839
8180,23866237,71.681849,67.194534,-4.487315,20.135996,71.549890,-0.131961,0.017414,70.523605,-1.158244,1.341529,69.756010,-1.925840,70.270507,-1.411342
8181,23708911,52.711066,55.754790,3.043726,9.264266,53.466724,0.755659,0.571020,52.470910,-0.240156,0.057675,53.897475,1.186409,53.326123,0.615057
8182,22168848,51.871575,50.325333,-1.546243,2.390866,51.601430,-0.270146,0.072979,50.723550,-1.148026,1.317965,50.883438,-0.988138,50.105712,-1.765864


In [25]:
for idx in range(len(dfs)):
    dfs[idx]['output_age_decon_1'] = age_debiasing(dfs[idx].output_age_1, dfs[idx].target_age)
    dfs[idx]['output_age_decon_2'] = age_debiasing(dfs[idx].output_age_2, dfs[idx].target_age) 
    dfs[idx]['output_age_decon_3'] = age_debiasing(dfs[idx].output_age_3, dfs[idx].target_age) 
    dfs[idx]['mean_output_age_decon'] = dfs[idx][['output_age_decon_1', 'output_age_decon_2', 'output_age_decon_3']].mean(axis=1)
    dfs[idx]['mean_age_delta_decon'] = dfs[idx]['mean_output_age_decon'] - dfs[idx]['target_age']

In [26]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon,output_age_decon_1,output_age_decon_2,output_age_decon_3,mean_output_age_decon,mean_age_delta_decon
0,23470825,72.307420,68.709730,-3.597688,12.943359,73.018036,0.710616,0.504975,73.407270,1.099852,1.209675,71.711679,-0.595741,72.360083,0.052662,69.937494,73.145594,73.803726,72.295605,-0.011816
1,23818742,57.839840,57.093216,-0.746624,0.557448,59.144497,1.304657,1.702129,57.383564,-0.456276,0.208188,57.873759,0.033919,57.574667,-0.265173,56.746321,59.069875,56.784144,57.533447,-0.306394
2,21725930,65.692352,62.648940,-3.043411,9.262348,64.478160,-1.214195,1.474268,64.320274,-1.372077,1.882596,63.815791,-1.876560,63.923556,-1.768796,63.055143,64.481266,64.151972,63.896127,-1.796225
3,24251841,67.121804,66.739975,-0.381829,0.145793,68.066060,0.944259,0.891626,69.167770,2.045967,4.185980,67.991268,0.869465,68.384932,1.263128,67.700732,68.121452,69.300740,68.374308,1.252504
4,21668564,75.721995,74.669510,-1.052485,1.107724,75.039734,-0.682261,0.465480,75.941560,0.219564,0.048208,75.216935,-0.505060,76.105347,0.383352,76.705143,75.196754,76.495522,76.132473,0.410479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,21345351,68.506507,69.871380,1.364877,1.862889,68.961060,0.454553,0.206618,70.393020,1.886514,3.558934,69.741820,1.235313,70.255346,1.748839,71.256610,69.029495,70.602140,70.296082,1.789575
8180,23866237,71.681849,67.194534,-4.487315,20.135996,71.549890,-0.131961,0.017414,70.523605,-1.158244,1.341529,69.756010,-1.925840,70.270507,-1.411342,68.216908,71.656052,70.740841,70.204600,-1.477249
8181,23708911,52.711066,55.754790,3.043726,9.264266,53.466724,0.755659,0.571020,52.470910,-0.240156,0.057675,53.897475,1.186409,53.326123,0.615057,55.226467,53.309360,51.566167,53.367331,0.656266
8182,22168848,51.871575,50.325333,-1.546243,2.390866,51.601430,-0.270146,0.072979,50.723550,-1.148026,1.317965,50.883438,-0.988138,50.105712,-1.765864,49.061028,51.416883,49.710209,50.062707,-1.808869


In [27]:
print("{:13} : {:14}  |  {:14}  |  {:13}  |   {:13}  |  {:}  |  {:} ".format('Modality', 'MAE (DECON)', 'MAE (DECON B)', "MSE (DECON)", "MSE (DECON B)",  "PR (DECON)", "PR (DECON B)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:.3f} ".format(
        names[idx],
        
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta_decon).mean(), np.abs(dfs[idx].mean_age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  MAE (DECON B)   |  MSE (DECON)    |   MSE (DECON B)  |  PR (DECON)  |  PR (DECON B) 
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.846 ± 0.007  |   0.864 ± 0.007   |   1.133 ± 0.018  |   1.183 ± 0.019   |   0.990   |   0.990 
T1_linear     :  0.934 ± 0.008  |   1.010 ± 0.008   |   1.367 ± 0.021  |   1.596 ± 0.025   |   0.989   |   0.987 
jacobian      :  1.894 ± 0.016  |   2.163 ± 0.018   |   5.628 ± 0.088  |   7.338 ± 0.115   |   0.955   |   0.943 
vbm           :  0.995 ± 0.008  |   1.018 ± 0.009   |   1.571 ± 0.025  |   1.643 ± 0.026   |   0.987   |   0.986 
T2_nonlinear  :  1.130 ± 0.010  |   1.174 ± 0.010   |   2.030 ± 0.032  |   2.191 ± 0.035   |   0.983   |   0.982 
T2_lesions    :  4.382 ± 0.036  |   4.399 ± 0.036   |   29.758 ± 0.458  |   29.975 ± 0.461   |   0.814   |   0.813 
swi           :  0.821 ± 0.007  |   0.934 ± 0.008   |   1.062 ± 0.017  |   1.375 ± 0.022   |   0.9